# Naive Bayes - Trabalho

<br><br>
<strong> Equipe </strong>:
1. Cristiano Filho
2. Dayana Rocha - 384349


## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.


## Respostas:

### Questão 1

#### 1.1 Definindo funções a serem utilizadas: 

In [384]:
# Carregar dados

import csv
 
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(0, len(dataset)):
        
        #Substituindo strings por números
        
        #Buying
        if(dataset[i][0] == 'vhigh'):    
            dataset[i][0] = 4
            
        elif(dataset[i][0] == 'high'):    
            dataset[i][0] = 3
            
        elif(dataset[i][0] == 'med'):    
            dataset[i][0] = 2
            
        elif(dataset[i][0] == 'low'):    
            dataset[i][0] = 1
        
        #Maint
        if(dataset[i][1] == 'vhigh'):    
            dataset[i][1] = 4
            
        elif(dataset[i][1] == 'high'):    
            dataset[i][1] = 3
            
        elif(dataset[i][1] == 'med'):    
            dataset[i][1] = 2
            
        elif(dataset[i][1] == 'low'):    
            dataset[i][1] = 1
        
        #Doors
        if(dataset[i][2] == '5more'):    
            dataset[i][2] = 6
        
        #Persons
        if(dataset[i][3] == 'more'):    
            dataset[i][3] = 6
            
        #Lug_boot
        if(dataset[i][4] == 'big'):    
            dataset[i][4] = 3
            
        elif(dataset[i][4] == 'med'):    
            dataset[i][4] = 2
            
        elif(dataset[i][4] == 'small'):    
            dataset[i][4] = 1
            
        #Safety
        if(dataset[i][5] == 'high'):    
            dataset[i][5] = 3
            
        elif(dataset[i][5] == 'med'):    
            dataset[i][5] = 2
            
        elif(dataset[i][5] == 'low'):    
            dataset[i][5] = 1
            
        #Classes
        if(dataset[i][6] == 'vgood'):    
            dataset[i][6] = 4
            
        elif(dataset[i][6] == 'good'):    
            dataset[i][6] = 3
            
        elif(dataset[i][6] == 'acc'):    
            dataset[i][6] = 2
            
        elif(dataset[i][6] == 'unacc'):    
            dataset[i][6] = 1
    
    
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    
    return dataset

In [385]:
# Dividir dataset em train e test

import random
def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

In [386]:
# Agrupar linhas por classe

def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [387]:
# Funções matemáticas que serão utilizadas

import math
def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [388]:
# Calcular desvio padrão e média por atributo

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [389]:
# "Summarizar" atributos por classes

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [390]:
# Calcular probabilidade de que um valor de atributo pertença à classe
# Função de densidade de probabilidade Gaussiana

def calculateProbability(x, mean, stdev):
    # No caso em que é zero, atribuir um valor grande para que o resultado se aproxime de zero
    if (stdev == 0):
        stdev = 10000
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * math.pow(stdev, 2))) * exponent

In [391]:
# Calcular probabilidade de atributo para cada classe

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [392]:
# Realizar predição

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [393]:
# Realizar predição para uma lista 

def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [394]:
# Calcular precisão da predição

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet[i][-1] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

#### 1.2 Algoritmo para realizar predição

In [396]:
# Arquivo do dataset
filename = 'carData.csv'

# Lendo arquivo
dataset = loadCsv(filename)

# Definindo coeficiente para divisão entre dados de treino e teste
splitRatio = 0.67

# Definindo conjuntos de treino e teste
train, test = splitDataset(dataset, splitRatio)

# Preparando modelo
summaries = summarizeByClass(train)

# Testando modelo
predictions = getPredictions(summaries, test)

# Calculando acurácia
accuracy = getAccuracy(test, predictions)
print('Accuracy: {0}%'.format(accuracy))

firstExpected = [linha[-1] for linha in test]
firstPredicted = predictions

Accuracy: 67.95096322241682%


## Questão 2

In [397]:
# Importando bibliotecas
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Lendo arquivo
dataset = loadCsv("carData.csv")

# Definindo coeficiente para divisão entre dados de treino e teste
splitRatio = 0.67

# Separando dados de treino e teste
train, test = splitDataset(dataset, splitRatio)

# Criando nosso modelo
model = GaussianNB()

# Treinando modelo
trainData = [linha[:-1] for linha in train]
trainTarget = [linha[-1] for linha in train]
model.fit(trainData, trainTarget)

# Fazendo predições
testData = [linha[:-1] for linha in test]
testTarget = [linha[-1] for linha in test]

secondExpected = testTarget
secondPredicted = model.predict(testData)

# Calculando acurácia
accuracy = accuracy_score(secondExpected, secondPredicted)
print('Accuracy: {0}%'.format(accuracy*100))

Accuracy: 69.35201401050787%


## Questão 3

#### 3.1 Análise de acurácia das duas metodologias

Em nossos testes, a acurácia se manteve no intervalo 64-76 para ambos algoritmos. Entretanto, vimos a segunda técnica ter um desempenho ligeiramente melhor, tendo, na maioria dos casos, a acurácia superior a 70. Por outro lado, a primeira técnica possui maior inclinação a gerar resultados com acurácia abaixo de 70. 

#### 3.2 Exposição de métricas

#### Abordagem 1

In [407]:
print(metrics.classification_report(firstExpected, firstPredicted))

              precision    recall  f1-score   support

         1.0       1.00      0.67      0.80       409
         2.0       0.52      0.81      0.63       117
         3.0       0.18      1.00      0.31        21
         4.0       0.00      0.00      0.00        24

   micro avg       0.68      0.68      0.68       571
   macro avg       0.42      0.62      0.43       571
weighted avg       0.83      0.68      0.71       571



#### Abordagem 2 (com sklearn)

In [406]:
print(metrics.classification_report(secondExpected, secondPredicted))

              precision    recall  f1-score   support

         1.0       0.89      0.84      0.87       403
         2.0       0.53      0.24      0.33       117
         3.0       0.46      0.21      0.29        29
         4.0       0.18      1.00      0.30        22

   micro avg       0.69      0.69      0.69       571
   macro avg       0.52      0.57      0.45       571
weighted avg       0.77      0.69      0.71       571

